<a href="https://colab.research.google.com/github/annalunde/NLSprintSummer2020/blob/master/NL_Sprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TO DO:
- importere all data: DONE
- finne signifikante parametere og dokumentere de som ikke er signifikante: DONE
- bygge validation, test og training set and write to new file: DONE
- normalisere all data: DONE
- les opp på ETL pipeline, ML biblio for Google og Feature Importance Permutation(?) og dropout layer: DONE
- bygge LSTM-modell, husk: noter kumulative/avg loss between every 10 epochs for training and validation data, if average loss is low on training but high on validation: indicates overfitting. Add dropout layer if it overfits: DONE
- kjøre try&fail metode på modellen med forksjellige parameterverdier og teste med validation set. Start with some different models that lapses in size: 50, 100, 200: DONE
- bygge en modell med alle parameterne: Kan se fil for hvilke som bidro mest, Feature Importance, Permutation: DROPPET, ettersom YR API'et ikke har de samme features
- Kjør modell med kun WP dataen og se om det er endring: DONE, ble ikke bedre
- Benytt Scikit learn sine funksjoner for cross-validation: DONE 
- utfør hyperparameter space search/randomized parameter search(?)/grid search: a way for us to set up parameters that can be tweaked, maybe do some random parameter search
- Sjekk om standardisering påvirker resultatet: DONE
- Få den live: sjekk notat





In [ ]:

import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

def import_weatherData():
      
      path = "/content/drive/My Drive/Colab Notebooks/data/weather_forecast_utc.csv"
      raw_dataset = pd.read_csv(path, header=0, #iterator=True, chunksize=10000, skiprows=range(1, 2000000),
                                usecols=[ 'datetime_forecast_utc', 'datetime_start_utc', 'SUB_AIR_PRESSURE', 'SUB_AIR_HUMIDITY', 'SUB_PRECIPITATION', 'SUB_CLOUD_COVER', 'windpark_zone'
                                         ]) #, skiprows=range(1, 20000)) #nrows = 2000000

      raw_dataset = raw_dataset.dropna()

      return raw_dataset




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def import_historicalData():
      
      path = "/content/drive/My Drive/Colab Notebooks/data/data.2018-10-31.93b9dcde-5e2e-11ea-8d9e-000d3a64d565.csv"
      raw_dataset = pd.read_csv(path, header=0, usecols=['timestamp', 
                                                         'ActivePower (Average)','WindSpeed (Average)'], sep=",", nrows = 2002500)
      

      raw_dataset = raw_dataset.dropna()

      return raw_dataset

In [ ]:
#Check autocorrelation in weather data
hist_data = import_historicalData()
output = hist_data['ActivePower (Average)'].autocorr(lag=1)
print(output)

0.953996527263065


In [ ]:
def import_runtimePlan():

      path = "/content/drive/My Drive/Colab Notebooks/data/running_plan.2018-10-31.93b9dcde-5e2e-11ea-8d9e-000d3a64d565.csv"
      raw_dataset = pd.read_csv(path, header=0, usecols=[
                                                         #'turbine','timestamp',
                                                    'ActivePowerLimit'
                                                    #, 'StateRun'
                                                    ], nrows = 100000)
      raw_dataset = raw_dataset.dropna()

      return raw_dataset

In [ ]:
from sklearn import preprocessing
import numpy as np

def normalize(data):
  #x_array = np.array(data[colName])
  #Normalize the data, has to be in NP array format
  return preprocessing.normalize([data])



In [ ]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from scipy.stats import pearsonr


def find_pearsonCorr(x,y):
# pearson's correlation feature selection for numeric input and numeric output
  corr, _ = pearsonr(x, y)
  print('Pearsons correlation: %.3f' % corr)


In [ ]:
from scipy.stats import spearmanr

def find_spearmanCorr(x,y):
# calculate spearman's correlation
  corr, _ = spearmanr(x, y)
  print('Spearmans correlation: %.3f' % corr)


Dokumentasjon:
- Vi har valgt å benytte forecasts to timer frem i tid
- Antar at parken ikke er for stor, bruker kun "WP": whole park, derfor predikerer vi også kun for hver turbin
- Totalt 47 turbiner, Viktig for å regne ut energi: Note that the dataset contains sensor data for Power (P) for each turbine, measured in KW (Kilowatts), while you will have to predict Energy (E), measured in KWh (Kilowatt-hours), for the whole park. In a time period with constant power, Energy can be calculated by the formula:

  E = P * t, where P is Power in KW, t is time in hours.

  One should summarize all turbines to get the overall energy.


In [ ]:
from datetime import datetime
import csv
from datetime import timedelta


def right_timedates(datetime_startIn, datetime_forecastIn):
      #Sjekker når værmeldingen er for og returnerer tidspunktet for en time senere
      datetime_start = str(datetime_startIn)
      datetime_forecast = str(datetime_forecastIn)
      datetime_start = datetime.strptime(datetime_start, '%Y-%m-%d %H:%M:%S+00:00')
      datetime_forecast = datetime.strptime(datetime_forecast, '%Y-%m-%d %H:%M:%S+00:00')
      diff = datetime_start - datetime_forecast
      return  diff  <= timedelta(hours = 1)  
    

In [ ]:
import numpy as np
def checkCorrelation_weather_and_histdata(weather_data, historical_data):
  #Dok: Trenger weather_data: datetime_forecast_utc, datetime_start_utc, SUB_WIND_SPEED_10
                #historical_data: timestamp, WindSpeed (Average), ActivePower (Average)
  list_weather = []
  list_hist = []  

  #for row_w in weather_data:
  for start_w in range (0,len(weather_data)-1):
    row_w = weather_data.iloc[start_w:start_w+1]
    if (row_w["windpark_zone"][start_w] == "WP"):

      if (right_timedates(row_w["datetime_start_utc"][start_w],row_w["datetime_forecast_utc"][start_w])):
        path = "/content/drive/My Drive/Colab Notebooks/data/data.2018-10-31.93b9dcde-5e2e-11ea-8d9e-000d3a64d565.csv"
        df = pd.read_csv(path, header=0, usecols=['timestamp', 'ActivePower (End)','WindSpeed (Average)'], sep=",", nrows = 2002500)
        df = df.dropna()

        start_date = str(row_w["datetime_start_utc"][start_w]) 
        mask = (df["timestamp"] == start_date)

        df = df.loc[mask]
        df.set_index("timestamp", inplace = True) 

        if (df.empty):
          continue

        #for row_h in historical_data:
        for start in range(0,len(df)-1):
          row_h = df.iloc[start:start+1]

          #Vi har en prediksjon to timer fram i tid og historical og weather er for ca. samme tidspunkt
          weather_el = row_w["SUB_CLOUD_COVER"][start_w]
          #float(row_w["SUB_CLOUD_COVER"][start_w] #"".replace(',',''))
          list_weather.append(weather_el)
          element = round(row_h["ActivePower (End)"][start_date],1)
          list_hist.append(element)
          if (len(list_weather) == 1000):
            print(list_weather)
            print(list_hist)
            normd_x = np.asarray(list_hist)
            normd_y = np.asarray(list_weather)
            normd_weather = normalize(normd_y)
            normd_historical = normalize(normd_x)
            normd_historical = normd_historical[0]
            normd_weather = normd_weather[0]
            find_pearsonCorr(normd_weather,normd_historical)
            find_spearmanCorr(normd_weather,normd_historical)
            return

      else:
        continue
            
  


When checking for correlation in the above sets, whe have not factored in the Active Power Limit. This will be neccessary to fix before building the model. This is fixed under. Also, we try to implement ETL pipelining to some extent, so we end up writing to a new CSV file when we build the data sets under.

In [ ]:
hist_data = import_historicalData()
weather_data = import_weatherData()

checkCorrelation_weather_and_histdata(weather_data, hist_data)

In [ ]:
import pandas as pd
drive.mount("drive", force_remount=True)

def build_training_data_file(weather_data, historical_data):
  #Bygger/skriver til filen som skal senere brukes til å dele inn i training, test og validation set
  
  #viktig: sjekk at filen er tom før skriver til den 
  #kan increase forecast tiden
 
  #Dok: Trenger weather_data: datetime_forecast_utc, datetime_start_utc, SUB_AIR_PRESSURE, SUB_AIR_HUMIDITY, SUB_PRECIPITATION, SUB_CLOUD_COVER
                #historical_data: timestamp, WindSpeed (Average), ActivePower (Average), ActivePowerLimit, StateRun
 
  init_write = 0
  #for row_w in weather_data:
  for start_w in range (0, len(weather_data)):
    row_w = weather_data.iloc[start_w:start_w+1]
    if (not(row_w.empty)):

      if (right_timedates(row_w["datetime_start_utc"][start_w],row_w["datetime_forecast_utc"][start_w])):
        path = "/content/drive/My Drive/Colab Notebooks/data/data.2018-10-31.93b9dcde-5e2e-11ea-8d9e-000d3a64d565.csv"
        df = pd.read_csv(path, header=0, usecols=['timestamp', 'ActivePower (Average)','WindSpeed (Average)'], sep=",") #nrows = 2002500)
        df = df.dropna()

        start_date = str(row_w["datetime_start_utc"][start_w])
        #viktig: må også sjekke at active power ikke er like active power limit (uten aggregate) og at stateRun = 1
        mask = (df["timestamp"] == start_date) & (df['StateRun'] == 1) & (df['ActivePower (Average)'] != df['ActivePowerLimit'])

        df = df.loc[mask]
        df.set_index("timestamp", inplace = True) 

        if (df.empty):
          continue
        
        print(row_w)

        #for row_h in historical_data:
        for start in range(0,len(df)):
          row_h = df.iloc[start:start+1]
          print(row_h,"row h")

          #Vi har en prediksjon to timer fram i tid og historical og weather er for ca. samme tidspunkt, staterun = 1 og makser ikke limit på power
          path = "/content/drive/My Drive/Colab Notebooks/data/cleaned_data_withoutCloud.csv"
          #format: SUB_AIR_PRESSURE, SUB_AIR_HUMIDITY, SUB_PRECIPITATION, //SUB_CLOUD_COVER//, WindSpeed (Average), ActivePower (Average)
          if (init_write == 0):
            element = [row_w["SUB_AIR_PRESSURE"][start_w],row_w["SUB_AIR_HUMIDITY"][start_w] ,row_w["SUB_PRECIPITATION"][start_w]  , row_h["WindSpeed (Average)"][start_date], row_h["ActivePower (Average)"][start_date]] 
            print(element, "element")
            #element = [row_w["SUB_AIR_PRESSURE"][start_w],row_w["SUB_AIR_HUMIDITY"][start_w] ,row_w["SUB_PRECIPITATION"][start_w] , row_w["SUB_CLOUD_COVER"][start_w] , row_h["WindSpeed (Average)"][start_date], row_h["ActivePower (Average)"][start_date]] 
            insert_df = pd.DataFrame([element], columns=['SUB_AIR_PRESSURE', 'SUB_AIR_HUMIDITY', 'SUB_PRECIPITATION', 'WindSpeed (Average)', 'ActivePower (Average)'])
            insert_df.to_csv(path, mode = 'a', index = False) 
            init_write = 1
          else:
            element = [row_w["SUB_AIR_PRESSURE"][start_w],row_w["SUB_AIR_HUMIDITY"][start_w] ,row_w["SUB_PRECIPITATION"][start_w] , row_h["WindSpeed (Average)"][start_date], row_h["ActivePower (Average)"][start_date]] 
            insert_df = pd.DataFrame([element], columns=['SUB_AIR_PRESSURE', 'SUB_AIR_HUMIDITY', 'SUB_PRECIPITATION', 'WindSpeed (Average)', 'ActivePower (Average)'])
            insert_df.to_csv(path, mode = 'a', header = False, index = False) 
                
          
          if (start_w == len(weather_data)-1):
            return
          
          else:
            continue 

      else:
        continue




Mounted at drive


In [ ]:
weather_data = import_weatherData()
historical_data = import_historicalData()
build_training_data_file(weather_data, historical_data)

In [ ]:
#for å splitte i test og train og normalisere data'en
from sklearn.model_selection import train_test_split
drive.mount("drive", force_remount=True)
import numpy as np
path = "/content/drive/My Drive/Colab Notebooks/data/CleanedData/cleaned_data_whole.csv"
df = pd.read_csv(path,  engine='python')
train, test = train_test_split(df, test_size=0.2, shuffle = False)

X_train = train.drop(['ActivePower (Average)'], axis=1)
y_train = train[['ActivePower (Average)']]
X_test = test.drop(['ActivePower (Average)'], axis=1)
y_test = test[['ActivePower (Average)']]


y_test = y_test.values #returns a numpy array
y_train = y_train.values
min_max_scaler_y = preprocessing.MinMaxScaler()
y_test_scaled = min_max_scaler_y.fit_transform(y_test)
y_train_scaled = min_max_scaler_y.fit_transform(y_train)
y_test = pd.DataFrame(y_test_scaled)
y_train = pd.DataFrame(y_train_scaled)

#convert air pressure to values
X_train["SUB_AIR_PRESSURE"] = X_train["SUB_AIR_PRESSURE"].str.replace(',', '').astype(float)
X_test["SUB_AIR_PRESSURE"] = X_test["SUB_AIR_PRESSURE"].str.replace(',', '').astype(float)

X_train_np = X_train.values
X_test_np = X_test.values

#print(X_train_np, "train before norm")

min_max_scaler_x = preprocessing.MinMaxScaler()
x_train_normd = min_max_scaler_x.fit_transform(X_train_np)
x_test_normd = min_max_scaler_x.fit_transform(X_test_np)
x_train = pd.DataFrame(x_train_normd)
x_test = pd.DataFrame(x_test_normd)


print(x_train)
print(x_test)

#print(x_train.values.shape)
#print(y_train.shape)
#print(x_test.values.shape)
#print(y_test.shape)
#print(isinstance(x_train.values, np.ndarray))
#time steps = 5
#input sequence = 1
#batch size = 447791
#(447791, 5, 1)


x_train = x_train.values.reshape((447796, 1, 5))
x_test = x_test.values.reshape((111949, 1, 5))
y_train = y_train.values.reshape((447796, 1, 1))
y_test = y_test.values.reshape((111949, 1, 1))




In [ ]:
#Save the scalers

import joblib
scalerX_filename = "/content/drive/My Drive/Colab Notebooks/data/scalerX.save"
scalerY_filename = "/content/drive/My Drive/Colab Notebooks/data/scalerY.save"
joblib.dump(min_max_scaler_x, scalerX_filename) 
joblib.dump(min_max_scaler_y, scalerY_filename) 

# And now to load...

scaler = joblib.load(scaler_filename) 


In [ ]:
#for å splitte i test og train og STANDARDISERE data'en
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
drive.mount("drive", force_remount=True)
import numpy as np
path = "/content/drive/My Drive/Colab Notebooks/data/CleanedData/cleaned_data_whole.csv"
df = pd.read_csv(path,  engine='python')
df = df.dropna()

X = df.drop(['ActivePower (Average)'], axis=1)
y = df[['ActivePower (Average)']]

#convert air pressure to values
X["SUB_AIR_PRESSURE"] = X["SUB_AIR_PRESSURE"].str.replace(',', '').astype(float)

values_X = X.values
values_y = y.values
# train the standardization
scaler_X = StandardScaler()
scaler = scaler_X.fit(values_X)
scaler_y = StandardScaler()
scaler = scaler_y.fit(values_y)
# standardization the dataset 
y = scaler_y.transform(values_y)


X = scaler_X.transform(values_X)

X = pd.DataFrame(X)
y = pd.DataFrame(y)

print(X)
print(y)

x_train = X.values.reshape((559739, 1, 5))
y_train = y.values.reshape((559739, 1, 1))

# inverse transform and print the first 5 rows
#inversed = scaler.inverse_transform(normalized)

bygge LSTM-modell, husk: noter kumulative/avg loss between every 10 epochs for training and validation data, if average loss is low on training but high on validation: indicates overfitting. Add dropout layer if it overfits: DONE

In [ ]:
import keras.backend as K
import tensorflow as tf
accepted_diff = 0.01
def linear_regression_equality(y_true, y_pred):
    diff = K.abs(y_true-y_pred)
    return K.mean(K.cast(diff < accepted_diff, tf.float32))

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import io
from keras.layers import Dropout
import h5py



from google.colab import drive
drive.mount('/content/drive')

#defining a model to use
model = tf.keras.Sequential()

#LSTM layer
model.add(tf.keras.layers.LSTM(200, activation='sigmoid',input_shape=(1, 5)))
#Dense layer
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
#compiling model 
model.compile(optimizer='adam', loss='mse', metrics=[linear_regression_equality])

# To avoid overfitting
mc = tf.keras.callbacks.ModelCheckpoint("/content/drive/My Drive/Colab Notebooks/data/best_model", monitor='val_linear_regression_equality', mode='max', save_best_only=True)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose = 2)

model.fit(x_train, y_train, epochs = 1000, batch_size =50,validation_split=0.2,  verbose = 2, callbacks=[early_stop, tfdocs.modeling.EpochDots(), mc])




In [ ]:
#To Export the model to serving format
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow import keras
import h5py


from keras.models import load_model
saved_model =  tf.keras.models.load_model("/content/drive/My Drive/Colab Notebooks/data/best_model_whole3.h5", custom_objects={'linear_regression_equality': linear_regression_equality})

import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}


In [ ]:
#Cross validation:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasRegressor

import numpy as np
from google.colab import drive
drive.mount('/content/drive')


import tensorflow as tf
from tensorflow import keras
import h5py

from keras.models import load_model

path = "/content/drive/My Drive/Colab Notebooks/data/cleaned_data_whole.csv"
df = pd.read_csv(path)
df = df.dropna()


X = df.drop(['ActivePower (Average)'], axis=1)
y = df[['ActivePower (Average)']]

y = y.values #returns a numpy array
min_max_scaler_y = preprocessing.MinMaxScaler()
y_scaled = min_max_scaler_y.fit_transform(y)
y = pd.DataFrame(y_scaled)

#convert air pressure to values
X["SUB_AIR_PRESSURE"] = X["SUB_AIR_PRESSURE"].str.replace(',', '').astype(float)

X = X.values
min_max_scaler_x = preprocessing.MinMaxScaler()
X = min_max_scaler_x.fit_transform(X)
X = pd.DataFrame(X)


#X = X.values.reshape((559739, 1, 5))
#y = y.values.reshape((559739, 1, 1))

def create_model():
  model = Sequential()
  model.add(tf.keras.layers.LSTM(200, activation='sigmoid',input_shape=(1, 5)))
  model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
  model.compile(optimizer='adam', loss='mse', metrics=[linear_regression_equality])
  return model

seed = 7
np.random.seed(seed)

# evaluate using 10-fold cross validation
model = KerasRegressor(build_fn=create_model, epochs=150, batch_size=10, verbose=2)

from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

model = KerasRegressor(build_fn=create_model, epochs=150, batch_size=10, verbose=0)

results = cross_val_score(model, X, y, cv=kfold)
print(results.mean())




In [ ]:
# Example on loading and predicting further on a saved model
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow import keras
import h5py


from keras.models import load_model
saved_model =  tf.keras.models.load_model("/content/drive/My Drive/Colab Notebooks/data/best_model_whole3.h5", custom_objects={'linear_regression_equality': linear_regression_equality})

arr = np.asarray([[0.5833,0.9166,0.,0.75,0.6668]])
arr = arr.reshape((1,1,5))
print(arr)
#_, train_acc = saved_model.evaluate(trainX, trainy, verbose=0)



pred = saved_model.predict(arr)
print(pred)
print(min_max_scaler_y.inverse_transform(pred))


print(saved_model.evaluate(x_test, y_test))




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[[0.5833 0.9166 0.     0.75   0.6668]]]
[[0.99928606]]
[[4006.7896]]
3499/3499 [==============================] - 5s 2ms/step - loss: nan - linear_regression_equality: 0.3799
[nan, 0.37988531589508057]


In [ ]:
LSTM: sigmoid
Dense: sigmoid
[[0.9998485]]
Epoch 143/10000
.7165/7165 - 13s - loss: 0.0025 - linear_regression_equality: 0.5062 - val_loss: 0.0025 - val_linear_regression_equality: 0.5076

LSTM: relu
Dense: sigmoid
Epoch 00019: early stopping
[[0.72211695]]
3499/3499 [==============================] - 4s 1ms/step - loss: 0.1131 - linear_regression_equality: 0.0084
[0.11307872086763382, 0.008377393707633018]

LSTM: sigmoid
Dense: linear
Epoch 00041: early stopping
[[0.96833193]]
3499/3499 [==============================] - 4s 1ms/step - loss: 0.0029 - linear_regression_equality: 0.3346
[0.0029051369056105614, 0.33458369970321655]

CONCLUSION: use sigmoid on both!

LSTM: sigmoid
Dense: sigmoid
Dropout: 0.2
Epoch 00016: early stopping
[[0.9976999]]
3499/3499 [==============================] - 5s 1ms/step - loss: 0.0315 - linear_regression_equality: 0.1502
[0.03149344399571419, 0.15018576383590698]

LSTM: sigmoid
Dense: sigmoid
Units = 50
[[0.9992273]]
Epoch 86/10000
.7165/7165 - 15s - loss: 0.0024 - linear_regression_equality: 0.5814 - val_loss: 0.0024 - val_linear_regression_equality: 0.5764

LSTM: sigmoid
Dense: sigmoid
Units = 50
Dropout: 0.2
Epoch 00016: early stopping
[[0.9966503]]
3499/3499 [==============================] - 5s 1ms/step - loss: 0.0320 - linear_regression_equality: 0.1529
[0.032032109797000885, 0.15288296341896057]

LSTM: sigmoid
Dense: sigmoid
Dropout: 0.2 
Units = 100
Epoch 00017: early stopping
[[0.9976829]]
3499/3499 [==============================] - 5s 1ms/step - loss: 0.0344 - linear_regression_equality: 0.1450
[0.034427348524332047, 0.14495213329792023]

LSTM: sigmoid
Dense: sigmoid
Units = 200
Epoch 92/10000
.7165/7165 - 34s - loss: 0.0024 - linear_regression_equality: 0.5903 - val_loss: 0.0024 - val_linear_regression_equality: 0.5995


WHEN ONLY USING WP DATA:
Val_reg_eq_limit: 0.025
LSTM: sigmoid
Dense: sigmoid
Units = 200
val_linear_regression_equality: 0.7995
&
Val_reg_eq_limit: 0.01
LSTM: sigmoid
Dense: sigmoid
Units = 200
Epoch 128/1000
.1434/1434 - 8s - loss: 0.0025 - linear_regression_equality: 0.5245 - val_loss: 0.0026 - val_linear_regression_equality: 0.5333
CONCLUSION: Use whole data set

Val_reg_eq_limit: 0.01, Saved model: whole2
LSTM: sigmoid
Dense: sigmoid
Epoch 61/1000
.7165/7165 - 30s - loss: 0.0031 - linear_regression_equality: 0.5346 - val_loss: 0.0026 - val_linear_regression_equality: 0.6290
Epoch 00061: early stopping, used whole data set

Val_reg_eq_limit: 0.01, Saved model: whole3
LSTM: sigmoid
Dense: sigmoid
Epoch 61/1000
.7165/7165 - 30s - loss: 0.0031 - linear_regression_equality: 0.5346 - val_loss: 0.0026 - val_linear_regression_equality: 0.64
Epoch 00061: early stopping, used whole data set

GIKK FRA Å NORMALISERE TIL Å STANDARDISERE DATAEN: Det ble mye dråligere, dropper det


In [ ]:
import pandas as pd

def import_yrData():
      
      path_oslo = "https://api.met.no/weatherapi/locationforecast/2.0/complete?lat=59.91&lon=10.71"
      path_stockholm = "https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=59.33&lon=18.07"
      path_koben = "https://api.met.no/weatherapi/locationforecast/2.0/complete?lat=55.68&lon=12.57"
      path_munich = "https://api.met.no/weatherapi/locationforecast/2.0/complete?lat=48.14&lon=11.59"
      
      raw_dataset = pd.read_json(path_stockholm, orient ='columns')

      #raw_dataset = raw_dataset.dropna()

      return raw_dataset

In [ ]:
from pandas import Timestamp
data = import_yrData()

import pandas as pd
from datetime import datetime
from datetime import timedelta

print(data)
print(data.loc["timeseries"][2])
print(data.loc["timeseries"][2][6])
print(data.loc["timeseries"][2][6]['time'])
print(data.loc["timeseries"][2][4]['data']['instant']['details'])

#må hente ut riktig tidspunkt 
a = True
init = 0
while (a):
  for i in range(1,1000):
    tid = datetime.now().strftime("%Y-%m-%d %H:%M")
    ut = data.loc["timeseries"][2][i]['time']
    ut = ut.replace("T", " ").replace(":00Z", "")
    
    ut = datetime.strptime(ut, '%Y-%m-%d %H:%M')
    tid = datetime.strptime(tid, '%Y-%m-%d %H:%M')
    diff = ut - tid
    if (diff <= timedelta(hours = 1)):
      for k in range (1,49):
          el_tid = data.loc["timeseries"][2][k]['time'].replace("T", " ").replace(":00Z", "")
          el = data.loc["timeseries"][2][k]["data"]["instant"]["details"]
          element_wind = el["wind_speed"]
          element_cloud = el["cloud_area_fraction"]
          el_p = data.loc["timeseries"][2][k]["data"]["next_1_hours"]["details"]
          element_precip = el_p["precipitation_amount"]
          #element_precip = (el_p["precipitation_amount_max"] + el_p["precipitation_amount_min"])/2
          element_press = el["air_pressure_at_sea_level"]
          element_humid = el["relative_humidity"]

          element = [el_tid,element_press, element_humid, element_precip,element_cloud, element_wind]
          element_ws = [el_tid,element_wind]
          
          path = "/content/drive/My Drive/Colab Notebooks/data/yrAPI/yrAPI_Stockholm_5aug2.csv"
          path_wind = "/content/drive/My Drive/Colab Notebooks/data/yrAPI/5.aug/ws_Stockholm_5aug2.csv"

          if (init ==0):
            insert_df = pd.DataFrame([element], columns=['tid','SUB_AIR_PRESSURE', 'SUB_AIR_HUMIDITY', 'SUB_PRECIPITATION', 'SUB_CLOUD_COVER', 'WindSpeed (Average)'])
            insert_df.to_csv(path, mode = 'a', header = True, index = False)
            ws_insert = pd.DataFrame([element_ws], columns=['timestamp', 'WindSpeed (Average)'])
            ws_insert.to_csv(path_wind, mode = 'a', header = True, index = False)
            init = 1


          else: 
            insert_df = pd.DataFrame([element], columns=['tid','SUB_AIR_PRESSURE', 'SUB_AIR_HUMIDITY', 'SUB_PRECIPITATION', 'SUB_CLOUD_COVER', 'WindSpeed (Average)'])
            insert_df.to_csv(path, mode = 'a', header = False, index = False)
            ws_insert = pd.DataFrame([element_ws], columns=['timestamp', 'WindSpeed (Average)'])
            ws_insert.to_csv(path_wind, mode = 'a', header = False, index = False)
            if (k == 48):
              a = False

          

    
#SUB_AIR_PRESSURE,SUB_AIR_HUMIDITY,SUB_PRECIPITATION,SUB_CLOUD_COVER,WindSpeed (Average)


In [ ]:
#When predicting energy: remember to multiply by 47 and each turbine by their staterun factor

# load a saved model
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow import keras
import h5py


from keras.models import load_model
saved_model =  tf.keras.models.load_model("/content/drive/My Drive/Colab Notebooks/data/best_model_whole3.h5", custom_objects={'linear_regression_equality': linear_regression_equality})

path_yrAPI = "/content/drive/My Drive/Colab Notebooks/data/yrAPI/yrAPI_Stockholm_5aug2.csv"
df = pd.read_csv(path_yrAPI)

  #'tid','SUB_AIR_PRESSURE', 'SUB_AIR_HUMIDITY', 'SUB_PRECIPITATION', 'SUB_CLOUD_COVER', 'WindSpeed (Average)'
for index,row in df.iterrows():
  arr = np.asarray([[row['SUB_AIR_PRESSURE'],row['SUB_AIR_HUMIDITY'], row['SUB_PRECIPITATION'], row['SUB_CLOUD_COVER'], row['WindSpeed (Average)']]])
  min_max_scaler_x.fit_transform(arr)
  arr = arr.reshape((1,1,5))

  pred = saved_model.predict(arr)
  pred = min_max_scaler_y.inverse_transform(pred)

  pred_scaled = 47*pred
 
  #write to file
  el = round(pred_scaled[0][0],2)
  print(el)
  element = [row['tid'], el]
  path_pred = "/content/drive/My Drive/Colab Notebooks/data/Predictions/5.aug/predictions_Stockholm_5aug2.csv"
  insert_df = pd.DataFrame([element], columns= ['timestamp', 'energy'])
  insert_df.to_csv(path_pred, mode = 'a', index = False, header= False) 





In [ ]:
path_yrAPI = "/content/drive/My Drive/Colab Notebooks/data/yrAPI.csv"
df = pd.read_csv(path_yrAPI)


for index,row in df.iterrows():
  element = [row['tid'], row['WindSpeed (Average)']]
  print(element)
  pathws = "/content/drive/My Drive/Colab Notebooks/data/ws.csv"
  insert_df = pd.DataFrame([element], columns= ['timestamp', 'energy'])
  insert_df.to_csv(pathws, mode = 'a', index = False, header= False) 


In [ ]:
#To connect to Google Cloud Platform:
from google.colab import auth
auth.authenticate_user()
CLOUD_PROJECT = 'teak-sunup-257603'
BUCKET = 'gs://' + CLOUD_PROJECT + '-tf2-models'


In [ ]:
!gcloud config set project $CLOUD_PROJECT
!gsutil mb $BUCKET
print(BUCKET)

Updated property [core/project].
Creating gs://teak-sunup-257603-tf2-models/...
gs://teak-sunup-257603-tf2-models


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow import keras


from keras.models import load_model
saved_model =  tf.keras.models.load_model("/content/drive/My Drive/Colab Notebooks/data/best_model", custom_objects={'linear_regression_equality': linear_regression_equality})

saved_model.save(BUCKET + '/custom_prediction_routine/greenforecast_model', save_format='tf')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
INFO:tensorflow:Assets written to: gs://teak-sunup-257603-tf2-models/custom_prediction_routine/greenforecast_model/assets


In [ ]:
MODEL = 'greenforecast_model'
!gcloud ai-platform models create $MODEL \
 --regions=us-central1 \
 --enable-logging #\
 #--enable-console-logging

Using endpoint [https://ml.googleapis.com/]
Created ml engine model [projects/teak-sunup-257603/models/greenforecast_model].


In [ ]:
VERSION = 'v5'
MODEL_DIR = BUCKET + '/custom_prediction_routine/greenforecast_model'
!gcloud beta ai-platform versions create $VERSION \
  --model $MODEL \
  --origin $MODEL_DIR \
  --runtime-version=1.13 \
  --python-version=3.5 \
  --framework='tensorflow' \
  #--package-uris $BUCKET/custom_prediction_routine/my_custom_code-0.1.tar.gz \
  #--prediction-class predictor.MyPredictor


#    --package-uris gs://{BUCKET}/{PACKAGES_DIR}/custom-model-0.1.tar.gz \
#    --prediction-class=custom_prediction.CustomModelPrediction \
#    --service-account custom@project_id.iam.gserviceaccount.com

Using endpoint [https://ml.googleapis.com/]


In [ ]:
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [ ]:
import pandas as pd
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

instance = [0.5833,0.9166,0.,0.75,0.666]
instance = arr.reshape((1,1,5))
print(instance)

# Serialization
numpyData = {"array": instance}
encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)  
inst = json.dumps(np.array(instance).toList())

# use dump() to write array into file
print("Printing JSON serialized NumPy array")
print(encodedNumpyData)
print(type(encodedNumpyData))

test_predictions = predict_json(CLOUD_PROJECT, MODEL, inst)
#print(min_max_scaler_y.inverse_transform(test_predictions))



# Deserialization
print("Decode JSON serialized NumPy array")
decodedArrays = json.loads(encodedNumpyData)

finalNumpyArray = numpy.asarray(decodedArrays["array"])
print("NumPy Array")
print(finalNumpyArray)

